In [49]:
# Environment setup
import sys
from pathlib import Path

# Add src to path
workspace_root = Path.cwd()
sys.path.insert(0, str(workspace_root / 'src'))

print(f"Project root: {workspace_root}")
print(f"Python version: {sys.version}")
print("✓ Environment configured")

Project root: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment
Python version: 3.12.0 (v3.12.0:0fb18b02c8, Oct  2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]
✓ Environment configured


In [50]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime
import json
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Thesis pipeline utilities
from thesis_pipeline.io.parquet import read_parquet, write_parquet

# Plotting setup
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All imports successful")

✓ All imports successful


## 1. Load Filtered Data

Load comments and submissions with topic assignments from notebook 15b.

In [51]:
# Paths
topics_path = workspace_root / 'data' / '02_topics' / 'reddit'
output_path = workspace_root / 'data' / '03_stance' / 'reddit'
output_path.mkdir(parents=True, exist_ok=True)

print(f"Input: {topics_path}")
print(f"Output: {output_path}")

# Check files exist
print(f"\nValidating input files:")
print(f"  Comments (hybrid): {(topics_path / 'comments_with_topics_hybrid.parquet').exists()}")
print(f"  Submissions (hybrid): {(topics_path / 'submissions_with_topics_hybrid.parquet').exists()}")
print(f"  Topic definitions: {(topics_path / 'topic_definitions.json').exists()}")

Input: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/02_topics/reddit
Output: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit

Validating input files:
  Comments (hybrid): True
  Submissions (hybrid): True
  Topic definitions: True


In [52]:
# Load data from hybrid topic assignment
print("Loading filtered data from hybrid approach...")

df_comments = read_parquet(topics_path / 'comments_with_topics_hybrid.parquet')
df_submissions = read_parquet(topics_path / 'submissions_with_topics_hybrid.parquet')

with open(topics_path / 'topic_definitions.json', 'r') as f:
    topic_definitions = json.load(f)

print(f"\n✓ Loaded {len(df_comments):,} comments (hybrid filtered)")
print(f"✓ Loaded {len(df_submissions):,} submissions (with topics)")
print(f"✓ Loaded {len(topic_definitions)} topic definitions")

print(f"\nComment columns: {df_comments.columns.tolist()}")

# Note: Use 'thread_topic' column from hybrid approach
if 'thread_topic' in df_comments.columns:
    df_comments['topic_id'] = df_comments['thread_topic']
    print(f"\n✓ Using 'thread_topic' column for topic assignments")
elif 'topic_id' not in df_comments.columns:
    print(f"\n⚠️  Warning: No topic column found!")

print(f"\nTopic distribution:")
for topic_id, count in df_comments['topic_id'].value_counts().items():
    topic_label = topic_definitions[topic_id]['label']
    print(f"  {topic_label}: {count:,} comments")

Loading filtered data from hybrid approach...

✓ Loaded 86,974 comments (hybrid filtered)
✓ Loaded 923,431 submissions (with topics)
✓ Loaded 5 topic definitions

Comment columns: ['comment_id', 'submission_id', 'body', 'created_utc', 'parent_id', 'thread_topic', 'sim_climate_change', 'sim_donald_trump', 'sim_gun_control', 'sim_immigration', 'sim_vaccination']

✓ Using 'thread_topic' column for topic assignments

Topic distribution:
  Donald Trump: 53,238 comments
  Gun Control: 17,467 comments
  Immigration: 8,988 comments
  Climate Change: 7,048 comments
  Vaccination: 233 comments


## 2. Define Stance Categories

Define clear stance definitions for each topic.

In [53]:
# Define stance categories for each topic
stance_definitions = {
    'climate_change': {
        'topic_label': 'Climate Change',
        'target_position': 'Climate change is real, human-caused, and requires action',
        'pro': {
            'label': 'Pro (Favor)',
            'description': 'Supports climate action, acknowledges human-caused climate change, advocates for environmental policies',
            'examples': [
                'We need to transition to renewable energy now',
                'The science is clear - humans are causing global warming',
                'Carbon emissions must be reduced immediately'
            ]
        },
        'against': {
            'label': 'Against (Oppose)',
            'description': 'Denies climate change, opposes climate action, questions climate science',
            'examples': [
                'Climate change is a hoax',
                'The earth naturally goes through temperature cycles',
                'Environmental regulations hurt the economy'
            ]
        },
        'neutral': {
            'label': 'Neutral',
            'description': 'Neither supports nor opposes, asks questions, presents both sides, unclear stance',
            'examples': [
                'What do experts say about this?',
                'I\'m not sure what to believe',
                'There are arguments on both sides'
            ]
        }
    },
    'donald_trump': {
        'topic_label': 'Donald Trump',
        'target_position': 'Donald Trump as president and his policies',
        'pro': {
            'label': 'Pro (Support Trump)',
            'description': 'Supports Trump, defends his actions/policies, positive about his presidency',
            'examples': [
                'Trump is making America great again',
                'Finally a president who keeps his promises',
                'Best economy we\'ve had in decades under Trump'
            ]
        },
        'against': {
            'label': 'Against (Oppose Trump)',
            'description': 'Opposes Trump, criticizes his actions/policies, negative about his presidency',
            'examples': [
                'Trump is unfit for office',
                'His policies are hurting Americans',
                'He\'s an embarrassment to the country'
            ]
        },
        'neutral': {
            'label': 'Neutral',
            'description': 'Neither supports nor opposes, factual reporting, balanced view, unclear stance',
            'examples': [
                'Trump signed this executive order today',
                'Some policies I agree with, others I don\'t',
                'What are the details of this decision?'
            ]
        }
    },
    'gun_control': {
        'topic_label': 'Gun Control',
        'target_position': 'Stricter gun control and regulations',
        'pro': {
            'label': 'Pro (Support Gun Control)',
            'description': 'Supports stricter gun laws, advocates for regulations, emphasizes public safety',
            'examples': [
                'We need universal background checks',
                'Assault weapons should be banned',
                'Common sense gun laws save lives'
            ]
        },
        'against': {
            'label': 'Against (Oppose Gun Control)',
            'description': 'Opposes gun restrictions, defends Second Amendment rights, pro-gun ownership',
            'examples': [
                'Shall not be infringed',
                'Gun control only hurts law-abiding citizens',
                'More guns means more safety'
            ]
        },
        'neutral': {
            'label': 'Neutral',
            'description': 'Neither strongly pro nor anti gun control, balanced view, unclear stance',
            'examples': [
                'Both sides have valid points',
                'What regulations are being proposed?',
                'I support some measures but not others'
            ]
        }
    },
    'immigration': {
        'topic_label': 'Immigration',
        'target_position': 'More open/liberal immigration policies',
        'pro': {
            'label': 'Pro (Support Immigration)',
            'description': 'Supports immigrants, advocates for reform/pathways to citizenship, opposes harsh restrictions',
            'examples': [
                'We should welcome refugees',
                'Dreamers deserve a path to citizenship',
                'Immigration strengthens our economy'
            ]
        },
        'against': {
            'label': 'Against (Restrict Immigration)',
            'description': 'Opposes immigration, supports border wall/restrictions, anti-illegal immigration',
            'examples': [
                'Build the wall',
                'Illegal immigration is out of control',
                'We need to protect our borders'
            ]
        },
        'neutral': {
            'label': 'Neutral',
            'description': 'Neither strongly pro nor anti immigration, balanced view, unclear stance',
            'examples': [
                'We need legal immigration reform',
                'What are the economic impacts?',
                'There must be a middle ground'
            ]
        }
    },
    'vaccination': {
        'topic_label': 'Vaccination',
        'target_position': 'Vaccines are safe, effective, and should be widely used',
        'pro': {
            'label': 'Pro (Support Vaccination)',
            'description': 'Supports vaccines, advocates for vaccination, trusts medical science',
            'examples': [
                'Vaccines save lives',
                'The science is clear on vaccine safety',
                'Everyone should get vaccinated'
            ]
        },
        'against': {
            'label': 'Against (Anti-Vaccine)',
            'description': 'Opposes vaccines, questions vaccine safety, anti-mandate',
            'examples': [
                'Vaccines are dangerous',
                'Natural immunity is better',
                'No one should be forced to vaccinate'
            ]
        },
        'neutral': {
            'label': 'Neutral',
            'description': 'Neither strongly pro nor anti vaccine, questions/concerns, unclear stance',
            'examples': [
                'What are the long-term effects?',
                'I have some vaccines but not others',
                'More research is needed'
            ]
        }
    }
}

print("Defined stance categories for all topics:")
for topic_id, stance_def in stance_definitions.items():
    print(f"\n{stance_def['topic_label']}:")
    print(f"  Target: {stance_def['target_position']}")
    print(f"  Categories: Pro / Against / Neutral")

Defined stance categories for all topics:

Climate Change:
  Target: Climate change is real, human-caused, and requires action
  Categories: Pro / Against / Neutral

Donald Trump:
  Target: Donald Trump as president and his policies
  Categories: Pro / Against / Neutral

Gun Control:
  Target: Stricter gun control and regulations
  Categories: Pro / Against / Neutral

Immigration:
  Target: More open/liberal immigration policies
  Categories: Pro / Against / Neutral

Vaccination:
  Target: Vaccines are safe, effective, and should be widely used
  Categories: Pro / Against / Neutral


## 3. Sampling Strategy

Sample 100 comments per topic using stratified sampling to ensure diversity.

In [54]:
# Sampling parameters
SAMPLES_PER_TOPIC = 50
RANDOM_SEED = 73

print(f"Sampling strategy:")
print(f"  Target: {SAMPLES_PER_TOPIC} comments per topic")
print(f"  Total target: {SAMPLES_PER_TOPIC * len(topic_definitions)} comments")
print(f"  Random seed: {RANDOM_SEED}")

# Add text length for filtering
df_comments['text_length'] = df_comments['body'].str.len()

print(f"\nText length statistics:")
print(df_comments['text_length'].describe())

Sampling strategy:
  Target: 50 comments per topic
  Total target: 250 comments
  Random seed: 73

Text length statistics:
count    86974.000000
mean       323.510325
std        509.273857
min         10.000000
25%         76.000000
50%        165.000000
75%        367.000000
max      10109.000000
Name: text_length, dtype: float64


In [55]:
# Filter criteria for annotation quality
print("Applying filters for annotation quality...\n")

# Filter 1: Minimum length (exclude very short comments)
MIN_LENGTH = 50
print(f"Filter 1: Minimum length = {MIN_LENGTH} characters")
df_filtered = df_comments[df_comments['text_length'] >= MIN_LENGTH].copy()
print(f"  Remaining: {len(df_filtered):,} comments ({len(df_filtered)/len(df_comments)*100:.1f}%)")

# Filter 2: Exclude deleted/removed
print(f"\nFilter 2: Exclude [deleted] and [removed]")
df_filtered = df_filtered[
    ~df_filtered['body'].isin(['[deleted]', '[removed]'])
].copy()
print(f"  Remaining: {len(df_filtered):,} comments ({len(df_filtered)/len(df_comments)*100:.1f}%)")

# Filter 3: Exclude AutoModerator
if 'author' in df_filtered.columns:
    print(f"\nFilter 3: Exclude AutoModerator")
    df_filtered = df_filtered[
        df_filtered['author'] != 'AutoModerator'
    ].copy()
    print(f"  Remaining: {len(df_filtered):,} comments ({len(df_filtered)/len(df_comments)*100:.1f}%)")

print(f"\nFiltered corpus:")
print(f"  Original: {len(df_comments):,} comments")
print(f"  Filtered: {len(df_filtered):,} comments")
print(f"  Reduction: {(1 - len(df_filtered)/len(df_comments))*100:.1f}%")

print(f"\nComments per topic (after filtering):")
for topic_id, count in df_filtered['topic_id'].value_counts().items():
    topic_label = topic_definitions[topic_id]['label']
    print(f"  {topic_label}: {count:,} comments")

Applying filters for annotation quality...

Filter 1: Minimum length = 50 characters
  Remaining: 75,109 comments (86.4%)

Filter 2: Exclude [deleted] and [removed]
  Remaining: 75,109 comments (86.4%)

Filtered corpus:
  Original: 86,974 comments
  Filtered: 75,109 comments
  Reduction: 13.6%

Comments per topic (after filtering):
  Donald Trump: 42,726 comments
  Gun Control: 16,764 comments
  Immigration: 8,490 comments
  Climate Change: 6,899 comments
  Vaccination: 230 comments


In [56]:
# Stratified sampling per topic
print(f"Sampling {SAMPLES_PER_TOPIC} comments per topic...\n")

sampled_dfs = []

for topic_id, topic_info in topic_definitions.items():
    topic_label = topic_info['label']
    
    # Get comments for this topic
    topic_comments = df_filtered[df_filtered['topic_id'] == topic_id].copy()
    
    print(f"{topic_label}:")
    print(f"  Available: {len(topic_comments):,} comments")
    
    if len(topic_comments) < SAMPLES_PER_TOPIC:
        print(f"  WARNING: Only {len(topic_comments)} comments available (< {SAMPLES_PER_TOPIC} target)")
        sample = topic_comments.copy()
    else:
        # Stratified sampling by text length quartiles
        # This ensures diversity in comment length
        topic_comments['length_quartile'] = pd.qcut(
            topic_comments['text_length'], 
            q=4, 
            labels=['Q1', 'Q2', 'Q3', 'Q4'],
            duplicates='drop'
        )
        
        # Sample proportionally from each quartile
        sample = topic_comments.groupby('length_quartile', group_keys=False).apply(
            lambda x: x.sample(n=min(len(x), SAMPLES_PER_TOPIC // 4), random_state=RANDOM_SEED)
        )
        
        # If we don't have enough, randomly sample remainder
        if len(sample) < SAMPLES_PER_TOPIC:
            remaining_needed = SAMPLES_PER_TOPIC - len(sample)
            remaining_pool = topic_comments[~topic_comments.index.isin(sample.index)]
            additional = remaining_pool.sample(n=min(len(remaining_pool), remaining_needed), random_state=RANDOM_SEED)
            sample = pd.concat([sample, additional])
    
    print(f"  Sampled: {len(sample)} comments")
    
    sampled_dfs.append(sample)
    print()

# Combine all samples
df_annotation_set = pd.concat(sampled_dfs, ignore_index=True)

print(f"\n✓ Total sampled: {len(df_annotation_set):,} comments")
print(f"\nSamples per topic:")
for topic_id, count in df_annotation_set['topic_id'].value_counts().items():
    topic_label = topic_definitions[topic_id]['label']
    print(f"  {topic_label}: {count} comments")

Sampling 50 comments per topic...

Climate Change:
  Available: 6,899 comments
  Sampled: 50 comments

Donald Trump:
  Available: 42,726 comments
  Sampled: 50 comments

Gun Control:
  Available: 16,764 comments
  Sampled: 50 comments

Immigration:
  Available: 8,490 comments
  Sampled: 50 comments

Vaccination:
  Available: 230 comments
  Sampled: 50 comments


✓ Total sampled: 250 comments

Samples per topic:
  Climate Change: 50 comments
  Donald Trump: 50 comments
  Gun Control: 50 comments
  Immigration: 50 comments
  Vaccination: 50 comments


## 4. Prepare Annotation Dataset

Add submission context and prepare columns for annotation.

In [57]:
# Add submission context for annotation
print("Adding submission context...")

# Create submission lookup with title, selftext, and subreddit
submission_lookup = df_submissions.set_index('submission_id')[
    ['title', 'selftext', 'subreddit']
].to_dict('index')

df_annotation_set['submission_title'] = df_annotation_set['submission_id'].map(
    lambda x: submission_lookup.get(x, {}).get('title', 'N/A')
)

df_annotation_set['submission_selftext'] = df_annotation_set['submission_id'].map(
    lambda x: submission_lookup.get(x, {}).get('selftext', '')
)

df_annotation_set['subreddit'] = df_annotation_set['submission_id'].map(
    lambda x: submission_lookup.get(x, {}).get('subreddit', 'N/A')
)

print(f"✓ Added submission context")
print(f"  Submissions with selftext: {df_annotation_set['submission_selftext'].notna().sum()}")
print(f"  Avg selftext length: {df_annotation_set['submission_selftext'].str.len().mean():.0f} chars")

Adding submission context...
✓ Added submission context
  Submissions with selftext: 250
  Avg selftext length: 290 chars


In [72]:
# Add parent comment context
print("Adding parent comment context...")

# Check if parent_id column exists
if 'parent_id' in df_annotation_set.columns:
    # Create comment lookup for fetching parent bodies
    comment_lookup = df_comments.set_index('comment_id')['body'].to_dict()
    
    def get_parent_body(parent_id):
        if pd.isna(parent_id):
            return None
        # Parent IDs come in format "t1_xxxxx" or "t3_xxxxx"
        # t1 = comment, t3 = submission
        if isinstance(parent_id, str):
            if parent_id.startswith('t3_'):
                # Parent is the submission itself
                return "[This is a top-level comment replying to the submission]"
            elif parent_id.startswith('t1_'):
                # Parent is another comment - extract the comment_id (remove t1_ prefix)
                comment_id = parent_id[3:]  # Remove "t1_" prefix to match comment_id column
                return comment_lookup.get(comment_id, "[Parent comment not found]")
        return None
    
    df_annotation_set['parent_body'] = df_annotation_set['parent_id'].apply(get_parent_body)
    
    parent_found = df_annotation_set['parent_body'].notna().sum()
    top_level = (df_annotation_set['parent_body'] == "[This is a top-level comment replying to the submission]").sum()
    
    print(f"✓ Added parent comment context")
    print(f"  Top-level comments (reply to submission): {top_level}")
    print(f"  Reply comments with parent found: {parent_found - top_level}")
    print(f"  Parent not found/available: {len(df_annotation_set) - parent_found}")
else:
    print("  Warning: parent_id column not found in dataset")
    df_annotation_set['parent_body'] = None

Adding parent comment context...
✓ Added parent comment context
  Top-level comments (reply to submission): 93
  Reply comments with parent found: 157
  Parent not found/available: 0


In [88]:
# Add annotation columns
print("Preparing annotation columns...")

df_annotation_set['stance_label'] = None  # To be filled: 'pro', 'against', 'neutral'
df_annotation_set['annotation_confidence'] = None  # Optional: 'high', 'medium', 'low'
df_annotation_set['annotation_notes'] = ''  # Optional: Any notes
df_annotation_set['annotator'] = ''  # Annotator ID
df_annotation_set['annotation_date'] = None

# Add unique ID for tracking
df_annotation_set['annotation_id'] = [
    f"ANN_{i:05d}" for i in range(1, len(df_annotation_set) + 1)
]

print(f"✓ Added annotation columns")
print(f"\nAnnotation columns: {['annotation_id', 'stance_label', 'annotation_confidence', 'annotation_notes', 'annotator', 'annotation_date']}")

Preparing annotation columns...
✓ Added annotation columns

Annotation columns: ['annotation_id', 'stance_label', 'annotation_confidence', 'annotation_notes', 'annotator', 'annotation_date']


In [89]:
# Reorder columns for annotation workflow
annotation_cols = [
    'annotation_id',
    'topic_id',
    'submission_title',
    'submission_selftext',
    'parent_body',
    'body',
    'stance_label',
    'annotation_confidence',
    'annotation_notes',
    'annotator',
    'annotation_date',
    'comment_id',
    'submission_id',
    'parent_id',
    'subreddit',
    'text_length',
    'created_utc',
    'score',
    'author'
]

# Only include columns that exist
available_cols = [col for col in annotation_cols if col in df_annotation_set.columns]

df_annotation_set = df_annotation_set[available_cols].copy()

print(f"✓ Reordered columns for annotation")
print(f"\nFinal columns: {df_annotation_set.columns.tolist()}")

✓ Reordered columns for annotation

Final columns: ['annotation_id', 'topic_id', 'submission_title', 'submission_selftext', 'parent_body', 'body', 'stance_label', 'annotation_confidence', 'annotation_notes', 'annotator', 'annotation_date', 'comment_id', 'submission_id', 'parent_id', 'subreddit', 'text_length', 'created_utc']


## 5. Save Annotation Dataset

Save the prepared dataset and create annotation guidelines.

In [90]:
# Save annotation dataset
annotation_file = output_path / 'stance_annotation_dataset.parquet'
write_parquet(df_annotation_set, annotation_file)

print(f"✓ Saved annotation dataset: {annotation_file}")
print(f"  {len(df_annotation_set):,} comments ready for annotation")
print(f"  {df_annotation_set.memory_usage(deep=True).sum() / 1e6:.1f} MB")

✓ Wrote 250 rows to stance_annotation_dataset.parquet (0.2 MB)
✓ Saved annotation dataset: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_annotation_dataset.parquet
  250 comments ready for annotation
  0.5 MB


In [91]:
# Save stance definitions
stance_def_file = output_path / 'stance_definitions.json'

with open(stance_def_file, 'w') as f:
    json.dump(stance_definitions, f, indent=2)

print(f"✓ Saved stance definitions: {stance_def_file}")

✓ Saved stance definitions: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_definitions.json


In [92]:
# Reference the updated annotation guidelines (v2)
import shutil

# Path to the v2 guidelines
guidelines_v2_source = workspace_root / 'data' / '03_stance' / 'reddit' / 'stance_annotation_guidelines_v2.md'

# Copy to output directory for easy access
guidelines_file = output_path / 'stance_annotation_guidelines_v2.md'

if guidelines_v2_source.exists():
    print(f"✓ Using annotation guidelines v2: {guidelines_file}")
    print(f"  Copied from: {guidelines_v2_source}")
    
    # Display first few lines as preview
    with open(guidelines_file, 'r') as f:
        preview_lines = [next(f) for _ in range(15)]
    print(f"\n📄 Guidelines preview:")
    print("".join(preview_lines))
    print("   [... see full file for complete guidelines ...]")
else:
    print(f"⚠️  Warning: Guidelines v2 not found at {guidelines_v2_source}")
    print(f"   Please ensure stance_annotation_guidelines_v2.md exists in data/03_stance/reddit/")
    guidelines_file = None

✓ Using annotation guidelines v2: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_annotation_guidelines_v2.md
  Copied from: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_annotation_guidelines_v2.md

📄 Guidelines preview:
Stance Annotation Guidelines v2.0

Date: 2025-12-21
Dataset: 250 comments (50 per target) after embedding-based target retrieval
Task: Tri-class stance annotation (PRO / AGAINST / NEUTRAL) towards a fixed target proposition per target
Primary objective: Produce reliable stance labels suitable for daily aggregation into stance distributions \pi_{t,k} and downstream Esteban–Ray polarisation indices.

⸻

1. Scope and core principle

This annotation task labels stance, not sentiment. A comment can be rude, sarcastic, or emotional without expressing stance towards the target proposition.

Stance is defined only with respect to the target proposition for the assigned target k. If the comment

## 6. Preview Annotation Dataset

Show examples from the annotation dataset.

In [93]:
# Show sample annotations per topic
print("=" * 80)
print("ANNOTATION DATASET PREVIEW")
print("=" * 80)

for topic_id in df_annotation_set['topic_id'].unique():
    topic_label = topic_definitions[topic_id]['label']
    topic_samples = df_annotation_set[df_annotation_set['topic_id'] == topic_id].head(3)
    
    print(f"\n{'='*80}")
    print(f"{topic_label.upper()} - Sample Annotations")
    print(f"{'='*80}")
    
    for idx, (_, row) in enumerate(topic_samples.iterrows(), 1):
        print(f"\n[{idx}] ID: {row['annotation_id']}")
        print(f"    Submission: {row['submission_title'][:80]}...")
        if 'submission_selftext' in row.index and pd.notna(row['submission_selftext']) and row['submission_selftext'].strip():
            selftext_preview = str(row['submission_selftext'])[:150]
            print(f"    Thread body: {selftext_preview}...")
        print(f"    Subreddit: r/{row['subreddit']}")
        if 'parent_body' in row.index and pd.notna(row['parent_body']):
            parent_preview = str(row['parent_body'])[:100]
            print(f"    Parent: {parent_preview}...")
        print(f"    Comment ({row['text_length']} chars):")
        print(f"    \"{row['body'][:200]}...\"")
        print(f"    → Stance label: [TO BE ANNOTATED]")
        print("-" * 80)

ANNOTATION DATASET PREVIEW

CLIMATE CHANGE - Sample Annotations

[1] ID: ANN_00001
    Submission: I'll ask again, please ONLY TRUMP SUPPORTERS answer :)...
    Thread body: Hello r/Trumpsupporters.

While I don't agree, I can completely understand your reason for supporting Trump, once I stopped researching about Trump on...
    Subreddit: r/AskTrumpSupporters
    Parent: [This is a top-level comment replying to the submission]...
    Comment (157 chars):
    "Global Warming is a hoax created by liberal agenda to steal jobs form hard working americans and give it to illegals and Asians who abuse their work permits...."
    → Stance label: [TO BE ANNOTATED]
--------------------------------------------------------------------------------

[2] ID: ANN_00002
    Submission: Partisan polarization on climate change is worse than ever...
    Subreddit: r/politics
    Parent: [Parent comment not found]...
    Comment (140 chars):
    "You think it's bad now? Keep delaying climate action and a

In [87]:
# Show first few rows of dataset
print("\nFirst 5 rows of annotation dataset:")
df_annotation_set


First 5 rows of annotation dataset:


,annotation_id,topic_id,submission_title,submission_selftext,parent_body,body,stance_label,annotation_confidence,annotation_notes,annotator,annotation_date,comment_id,submission_id,parent_id,subreddit,text_length,created_utc
0,ANN_00001,climate_change,"I'll ask again, please ONLY TRUMP SUPPORTERS a...",Hello r/Trumpsupporters.\n\nWhile I don't agre...,[This is a top-level comment replying to the s...,Global Warming is a hoax created by liberal ag...,None,None,,,None,d7xpahd,53ypcs,t3_53ypcs,AskTrumpSupporters,157,1474565084
1,ANN_00002,climate_change,Partisan polarization on climate change is wor...,,[Parent comment not found],You think it's bad now? Keep delaying climate ...,None,None,,,None,d7fzteo,51l2mz,t1_d7cpx1e,politics,140,1473446950
2,ANN_00003,climate_change,"They're changing the AlGoreRhythm again: ""Al G...",[deleted],[This is a top-level comment replying to the s...,Call it 'carbon fraud' instead of 'climate cha...,None,None,,,None,d8eym7x,55zg0t,t3_55zg0t,The_Donald,51,1475674120
3,ANN_00004,climate_change,Club of Rome: “In Searching For A New Enemy To...,,[Parent comment not found],"And under the guise of fighting ""global warmin...",None,None,,,None,d82e1p4,54j0q1,t1_d82djkt,uncensorednews,113,1474876106
4,ANN_00005,climate_change,Clinton: We cannot put 'climate denier' in Whi...,,[Parent comment not found],Look let's be honest. Climate change is real. ...,None,None,,,None,d8o6tha,570a3y,t1_d8o6o1d,politics,162,1476235699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,ANN_00246,vaccination,Part 2: What's At Stake With the Proposed CDC ...,,[This is a top-level comment replying to the s...,"The excellent documentary film ""Vaxxed"" reveal...",None,None,,,None,d8nfqyl,56ymyx,t3_56ymyx,Libertarian,1294,1476200853
246,ANN_00247,vaccination,"Anti-rabies vaccine in short supply, patients ...",,[This is a top-level comment replying to the s...,134\n\n---\n\n&gt; * A number of patients who ...,None,None,,,None,d83wzs6,54po3o,t3_54po3o,willis7737_news,965,1474970099
247,ANN_00248,vaccination,"""Vaccines cause autism"" is Bullshit (Penn and ...",,There is a case that vaccines caused outbreak ...,[WHO disagrees](http://www.who.int/emergencies...,None,None,,,None,d7caebv,51iou9,t1_d7c9703,GoldandBlack,761,1473213278
248,ANN_00249,vaccination,California Today: Vaccinate Schoolchildren or ...,,[This is a top-level comment replying to the s...,Good. \n\nUnless they have a compromised immun...,None,None,,,None,d7d3hb7,51mn8u,t3_51mn8u,politics,146,1473272007


## 7. Interactive Annotation Interface

Use this interactive widget to annotate comments one by one.

In [83]:
# Install ipywidgets if needed (uncomment if not installed)
# !pip install ipywidgets

from ipywidgets import widgets, Layout, HTML, VBox, HBox, Button
from IPython.display import display, clear_output
import pandas as pd

print("✓ Interactive annotation interface ready")
print("\nRun the next cell to start annotating")

✓ Interactive annotation interface ready

Run the next cell to start annotating


In [67]:
# Interactive Annotation Widget
class StanceAnnotator:
    def __init__(self, df_annotation_set, topic_definitions, stance_definitions, output_path):
        self.df = df_annotation_set.copy()
        self.topic_definitions = topic_definitions
        self.stance_definitions = stance_definitions
        self.output_path = output_path
        self.current_idx = 0
        
        # Find first unannotated comment
        unannotated = self.df[self.df['stance_label'].isna()]
        if len(unannotated) > 0:
            self.current_idx = unannotated.index[0]
        
        # Create widgets
        self.setup_widgets()
        
    def setup_widgets(self):
        # Header
        self.header = HTML(value="<h3>Stance Annotation Interface</h3>")
        
        # Progress info
        self.progress_label = HTML(value="")
        
        # Comment display
        self.comment_display = HTML(value="", layout=Layout(width='95%', border='1px solid #ddd', padding='10px'))
        
        # Guidelines display
        self.guidelines_display = HTML(value="", layout=Layout(width='95%', border='1px solid #e8f4f8', padding='10px', background='#f0f8ff'))
        
        # Stance buttons
        self.pro_button = Button(description='✓ PRO', button_style='success', layout=Layout(width='150px', height='50px'))
        self.against_button = Button(description='✗ AGAINST', button_style='danger', layout=Layout(width='150px', height='50px'))
        self.neutral_button = Button(description='○ NEUTRAL', button_style='info', layout=Layout(width='150px', height='50px'))
        
        # Navigation buttons
        self.prev_button = Button(description='← Previous', button_style='', layout=Layout(width='120px'))
        self.next_button = Button(description='Next →', button_style='', layout=Layout(width='120px'))
        self.skip_button = Button(description='Skip', button_style='warning', layout=Layout(width='120px'))
        
        # Save button
        self.save_button = Button(description='💾 Save Progress', button_style='primary', layout=Layout(width='150px'))
        
        # Confidence selector
        self.confidence_dropdown = widgets.Dropdown(
            options=['', 'high', 'medium', 'low'],
            description='Confidence:',
            layout=Layout(width='200px')
        )
        
        # Notes field
        self.notes_text = widgets.Textarea(
            placeholder='Optional notes...',
            description='Notes:',
            layout=Layout(width='500px', height='60px')
        )
        
        # Status message
        self.status_label = HTML(value="")
        
        # Wire up events
        self.pro_button.on_click(lambda b: self.annotate('pro'))
        self.against_button.on_click(lambda b: self.annotate('against'))
        self.neutral_button.on_click(lambda b: self.annotate('neutral'))
        self.prev_button.on_click(lambda b: self.navigate(-1))
        self.next_button.on_click(lambda b: self.navigate(1))
        self.skip_button.on_click(lambda b: self.navigate(1))
        self.save_button.on_click(lambda b: self.save_annotations())
        
    def get_guidelines_html(self, topic_id):
        """Generate HTML for topic-specific guidelines"""
        stance_def = self.stance_definitions[topic_id]
        
        html = f"""
        <div style='background-color: #f0f8ff; padding: 10px; border-radius: 5px;'>
            <h4 style='color: #2c5aa0; margin-top: 0;'>{stance_def['topic_label']} - Stance Guidelines</h4>
            <p><strong>Target Position:</strong> {stance_def['target_position']}</p>
            
            <div style='margin: 10px 0;'>
                <p style='color: #28a745; font-weight: bold; margin: 5px 0;'>✓ PRO (Support):</p>
                <p style='margin: 5px 0 5px 20px;'>{stance_def['pro']['description']}</p>
                <p style='margin: 5px 0 5px 20px; font-style: italic; color: #666;'>
                    Examples: "{stance_def['pro']['examples'][0]}"
                </p>
            </div>
            
            <div style='margin: 10px 0;'>
                <p style='color: #dc3545; font-weight: bold; margin: 5px 0;'>✗ AGAINST (Oppose):</p>
                <p style='margin: 5px 0 5px 20px;'>{stance_def['against']['description']}</p>
                <p style='margin: 5px 0 5px 20px; font-style: italic; color: #666;'>
                    Examples: "{stance_def['against']['examples'][0]}"
                </p>
            </div>
            
            <div style='margin: 10px 0;'>
                <p style='color: #17a2b8; font-weight: bold; margin: 5px 0;'>○ NEUTRAL:</p>
                <p style='margin: 5px 0 5px 20px;'>{stance_def['neutral']['description']}</p>
                <p style='margin: 5px 0 5px 20px; font-style: italic; color: #666;'>
                    Examples: "{stance_def['neutral']['examples'][0]}"
                </p>
            </div>
        </div>
        """
        return html
    
    def display_comment(self):
        """Display current comment and guidelines"""
        if self.current_idx >= len(self.df):
            self.comment_display.value = "<p style='color: green; font-size: 18px;'>✓ All comments reviewed!</p>"
            self.guidelines_display.value = ""
            self.pro_button.disabled = True
            self.against_button.disabled = True
            self.neutral_button.disabled = True
            return
        
        row = self.df.iloc[self.current_idx]
        topic_id = row['topic_id']
        topic_label = self.topic_definitions[topic_id]['label']
        
        # Update progress
        annotated_count = self.df['stance_label'].notna().sum()
        total_count = len(self.df)
        progress_pct = (annotated_count / total_count * 100) if total_count > 0 else 0
        
        self.progress_label.value = f"""
        <div style='margin: 10px 0;'>
            <strong>Progress:</strong> {annotated_count} / {total_count} ({progress_pct:.1f}%)
            | <strong>Current:</strong> {self.current_idx + 1} / {total_count}
            | <strong>Topic:</strong> {topic_label}
        </div>
        """
        
        # Display comment
        current_label = row['stance_label'] if pd.notna(row['stance_label']) else 'Not annotated'
        label_color = {
            'pro': 'green',
            'against': 'red', 
            'neutral': 'blue'
        }.get(str(current_label).lower(), 'gray')
        
        # Build submission selftext section if available
        submission_body_section = ""
        if 'submission_selftext' in row.index and pd.notna(row['submission_selftext']) and row['submission_selftext'].strip():
            selftext = str(row['submission_selftext']).strip()
            # Truncate if very long
            display_selftext = selftext if len(selftext) <= 400 else selftext[:400] + "..."
            submission_body_section = f"""
            <div style='background-color: #fffacd; padding: 8px; margin: 10px 0; border-left: 4px solid #ffd700;'>
                <p style='margin: 0 0 5px 0; color: #666;'><strong>📄 Submission Body (Thread Context):</strong></p>
                <p style='margin: 0; white-space: pre-wrap; font-size: 13px; color: #555;'>{display_selftext}</p>
            </div>
            """
        
        # Build parent comment section if available
        parent_section = ""
        if 'parent_body' in row.index and pd.notna(row['parent_body']):
            parent_text = row['parent_body']
            if parent_text == "[This is a top-level comment replying to the submission]":
                parent_section = f"""
                <div style='background-color: #f0f0f0; padding: 8px; margin: 10px 0; border-left: 4px solid #999;'>
                    <p style='margin: 0; color: #666; font-style: italic;'>↑ Top-level comment (replies directly to submission)</p>
                </div>
                """
            else:
                # Truncate if very long
                display_parent = parent_text if len(parent_text) <= 300 else parent_text[:300] + "..."
                parent_section = f"""
                <div style='background-color: #f9f9f9; padding: 8px; margin: 10px 0; border-left: 4px solid #999;'>
                    <p style='margin: 0 0 5px 0; color: #666;'><strong>↑ Parent Comment (context):</strong></p>
                    <p style='margin: 0; white-space: pre-wrap; font-size: 13px; color: #555;'>{display_parent}</p>
                </div>
                """
        
        self.comment_display.value = f"""
        <div style='padding: 10px;'>
            <p><strong>ID:</strong> {row['annotation_id']} | 
               <strong>Current Label:</strong> <span style='color: {label_color}; font-weight: bold;'>{current_label}</span></p>
            
            <div style='background-color: #fff3cd; padding: 8px; margin: 10px 0; border-left: 4px solid #ffc107;'>
                <p style='margin: 0;'><strong>📌 Submission Title:</strong></p>
                <p style='margin: 5px 0;'>{row['submission_title']}</p>
                <p style='margin: 5px 0; color: #666;'><em>r/{row['subreddit']}</em></p>
            </div>
            
            {submission_body_section}
            
            {parent_section}
            
            <div style='background-color: #e7f3ff; padding: 10px; margin: 10px 0; border-left: 4px solid #2196F3;'>
                <p style='margin: 0 0 5px 0;'><strong>💬 Comment to Annotate:</strong></p>
                <p style='margin: 0; white-space: pre-wrap; font-size: 14px;'>{row['body']}</p>
            </div>
            
            <p style='color: #666; font-size: 12px;'>
                Length: {row['text_length']} chars | 
                Score: {row.get('score', 'N/A')} | 
                Author: {row.get('author', 'N/A')}
            </p>
        </div>
        """
        
        # Display topic-specific guidelines
        self.guidelines_display.value = self.get_guidelines_html(topic_id)
        
        # Update confidence and notes if already annotated
        if pd.notna(row['stance_label']):
            self.confidence_dropdown.value = row.get('annotation_confidence', '')
            self.notes_text.value = row.get('annotation_notes', '')
        else:
            self.confidence_dropdown.value = ''
            self.notes_text.value = ''
        
        # Enable buttons
        self.pro_button.disabled = False
        self.against_button.disabled = False
        self.neutral_button.disabled = False
    
    def annotate(self, stance):
        """Record annotation"""
        if self.current_idx >= len(self.df):
            return
        
        # Update dataframe
        self.df.at[self.df.index[self.current_idx], 'stance_label'] = stance
        self.df.at[self.df.index[self.current_idx], 'annotation_confidence'] = self.confidence_dropdown.value
        self.df.at[self.df.index[self.current_idx], 'annotation_notes'] = self.notes_text.value
        self.df.at[self.df.index[self.current_idx], 'annotation_date'] = datetime.now().strftime('%Y-%m-%d')
        
        # Show confirmation
        self.status_label.value = f"<span style='color: green;'>✓ Saved: {stance}</span>"
        
        # Auto-advance
        self.navigate(1)
    
    def navigate(self, direction):
        """Navigate to next/previous comment"""
        self.current_idx += direction
        
        # Bounds check
        if self.current_idx < 0:
            self.current_idx = 0
        elif self.current_idx >= len(self.df):
            self.current_idx = len(self.df) - 1
        
        self.status_label.value = ""
        self.display_comment()
    
    def save_annotations(self):
        """Save current progress"""
        try:
            output_file = self.output_path / 'stance_annotation_dataset_progress.parquet'
            write_parquet(self.df, output_file)
            
            annotated_count = self.df['stance_label'].notna().sum()
            self.status_label.value = f"<span style='color: green; font-weight: bold;'>✓ Saved! {annotated_count} annotations completed.</span>"
        except Exception as e:
            self.status_label.value = f"<span style='color: red;'>✗ Error saving: {e}</span>"
    
    def display(self):
        """Display the widget interface"""
        self.display_comment()
        
        # Layout
        stance_buttons = HBox([self.pro_button, self.against_button, self.neutral_button], 
                              layout=Layout(justify_content='center', margin='10px 0'))
        
        nav_buttons = HBox([self.prev_button, self.skip_button, self.next_button], 
                           layout=Layout(justify_content='center', margin='10px 0'))
        
        controls = HBox([self.confidence_dropdown, self.save_button], 
                       layout=Layout(justify_content='center', margin='10px 0'))
        
        interface = VBox([
            self.header,
            self.progress_label,
            self.comment_display,
            self.guidelines_display,
            stance_buttons,
            self.notes_text,
            controls,
            nav_buttons,
            self.status_label
        ])
        
        display(interface)

print("✓ StanceAnnotator class defined")
print("\nRun the next cell to start the annotation interface")

✓ StanceAnnotator class defined

Run the next cell to start the annotation interface


In [68]:
# Start the interactive annotation
annotator = StanceAnnotator(
    df_annotation_set=df_annotation_set,
    topic_definitions=topic_definitions,
    stance_definitions=stance_definitions,
    output_path=output_path
)

annotator.display()

## 7. Summary

Generate summary of annotation dataset.

In [97]:
# Create summary
summary = {
    'notebook': '16a_stance_annotation_sampling',
    'timestamp': datetime.now().isoformat(),
    'sampling': {
        'target_per_topic': SAMPLES_PER_TOPIC,
        'random_seed': RANDOM_SEED,
        'min_text_length': MIN_LENGTH,
        'strategy': 'stratified by text length quartiles'
    },
    'dataset': {
        'total_comments': len(df_annotation_set),
        'topics': len(topic_definitions),
        'samples_per_topic': df_annotation_set['topic_id'].value_counts().to_dict()
    },
    'text_statistics': {
        'mean_length': float(df_annotation_set['text_length'].mean()),
        'median_length': float(df_annotation_set['text_length'].median()),
        'min_length': int(df_annotation_set['text_length'].min()),
        'max_length': int(df_annotation_set['text_length'].max())
    },
    'outputs': {
        'annotation_dataset': str(annotation_file),
        'stance_definitions': str(stance_def_file),
        'guidelines': str(guidelines_file)
    }
}

# Save summary
summary_file = output_path / '16a_sampling_summary.json'
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Saved summary: {summary_file}")

✓ Saved summary: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/16a_sampling_summary.json


In [98]:
# Print final summary
print("\n" + "=" * 80)
print("ANNOTATION DATASET READY")
print("=" * 80)

print(f"\n✓ Sampled {len(df_annotation_set):,} comments for annotation")
print(f"\nSamples per topic:")
for topic_id, count in df_annotation_set['topic_id'].value_counts().items():
    topic_label = topic_definitions[topic_id]['label']
    print(f"  {topic_label}: {count} comments")

print(f"\n✓ Created files:")
print(f"  {annotation_file}")
print(f"  {stance_def_file}")
print(f"  {guidelines_file}")
print(f"  {summary_file}")

print("\n" + "=" * 80)
print("NEXT STEPS")
print("=" * 80)

print("\n1. MANUAL ANNOTATION:")
print("   - Read the annotation guidelines thoroughly")
print("   - Load the annotation dataset")
print("   - Fill in 'stance_label' column: 'pro', 'against', or 'neutral'")
print("   - Optionally add confidence and notes")
print("   - Save as 'stance_annotation_dataset_annotated.parquet'")

print("\n2. QUALITY CHECKS:")
print("   - Ensure all rows have stance_label filled")
print("   - Review a random sample for consistency")
print("   - If multiple annotators, calculate inter-annotator agreement")

print("\n3. PROCEED TO NOTEBOOK 16b:")
print("   - Use annotated dataset to train stance detection model")
print("   - Apply model to all filtered comments")
print("   - Validate on held-out annotations")

print("\n" + "=" * 80)


ANNOTATION DATASET READY

✓ Sampled 250 comments for annotation

Samples per topic:
  Climate Change: 50 comments
  Donald Trump: 50 comments
  Gun Control: 50 comments
  Immigration: 50 comments
  Vaccination: 50 comments

✓ Created files:
  /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_annotation_dataset.parquet
  /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_definitions.json
  /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/stance_annotation_guidelines_v2.md
  /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/03_stance/reddit/16a_sampling_summary.json

NEXT STEPS

1. MANUAL ANNOTATION:
   - Read the annotation guidelines thoroughly
   - Load the annotation dataset
   - Fill in 'stance_label' column: 'pro', 'against', or 'neutral'
   - Optionally add confidence and notes
   - Save as 'stance_annotation_dataset_annotated.parquet'

2. QUALITY CHECKS:
 